# Limp Explorer

Study population preflop limps from the normalized warehouse. The focus is on non-hero limps with revealed hole cards so we can map their range construction.

## Usage Notes

- Set `FORCE_RELOAD = True` after refreshing the warehouse or adjusting the loader to rebuild the cached event set.
- `MAX_TOP_COMBOS` and `MAX_POSITION_COMBOS` control how many combos are displayed in the summary tables.
- Ignition reveals all hole cards, but the notebook filters out hero actions and any seats without recorded cards. Results reflect only the population limps with visible holdings.

In [ ]:
from pathlib import Path
import os

def _locate_project_root() -> Path:
    current = Path().resolve()
    for candidate in (current, *current.parents):
        if (candidate / "AGENTS.md").exists():
            return candidate
    raise FileNotFoundError("Repository root not found from notebook location.")

PROJECT_ROOT = _locate_project_root()
del _locate_project_root

DB_CANDIDATES = [
    Path(r"T:\Dev\ignition\drivehud\drivehud.db"),
    Path("/mnt/t/Dev/ignition/drivehud/drivehud.db"),
    PROJECT_ROOT / "drivehud" / "drivehud.db",
]

for candidate in DB_CANDIDATES:
    if candidate.exists():
        DB_PATH = candidate
        break
else:
    checked = os.linesep.join(str(p) for p in DB_CANDIDATES)
    message = "Database not found. Checked:" + os.linesep + checked
    raise FileNotFoundError(message)

CACHE_PATH = PROJECT_ROOT / "analysis" / "cache" / "limp_events.json"

if not DB_PATH.exists():
    raise FileNotFoundError(f"Database not found at {DB_PATH}")


In [ ]:
import sys
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))


In [ ]:
import importlib
import pandas as pd
from IPython.display import display

import limp_utils
limp_utils = importlib.reload(limp_utils)

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.precision', 3)



In [ ]:
# --- Configuration ---
FORCE_RELOAD = True
MAX_TOP_COMBOS = 20
MAX_POSITION_COMBOS = 10
POSITION_ORDER = [
    'SB', 'BB', 'BTN', 'CO', 'HJ', 'LJ', 'MP', 'MP1', 'MP2', 'UTG+3', 'UTG+2', 'UTG+1', 'UTG',
    'Straddle', 'Unknown'
]


In [ ]:
events = limp_utils.load_limp_events(DB_PATH, cache_path=CACHE_PATH, force=FORCE_RELOAD)
resolved_db = limp_utils.get_drivehud_db_path()
if resolved_db:
    print(f'Using DriveHUD DB at: {resolved_db}')
else:
    print('DriveHUD DB path could not be resolved.')
limps_df = pd.DataFrame(events)

if limps_df.empty:
    print('No limps with revealed cards were found in the current dataset.')
else:
    classifications = [
        limp_utils.classify_preflop_hand(c1, c2)
        for c1, c2 in zip(limps_df['c1'], limps_df['c2'])
    ]

    limps_df['hand_label'] = [cls['label'] if cls else None for cls in classifications]
    limps_df['hand_group'] = [cls['group'] if cls else None for cls in classifications]
    limps_df['hi_rank'] = [cls['hi_rank'] if cls else None for cls in classifications]
    limps_df['lo_rank'] = [cls['lo_rank'] if cls else None for cls in classifications]
    limps_df['suited'] = [cls['suited'] if cls else None for cls in classifications]
    limps_df['hi_value'] = [cls['hi_value'] if cls else None for cls in classifications]
    limps_df['lo_value'] = [cls['lo_value'] if cls else None for cls in classifications]

    limps_df = limps_df.dropna(subset=['hand_label']).copy()

    limps_df['position_pre'] = limps_df['position_pre'].fillna('Unknown')
    limps_df['limp_type'] = limps_df['limp_type'].fillna('open')
    limps_df['only_blinds'] = limps_df['only_blinds'].fillna(False)
    limps_df['cards_shown'] = limps_df['cards_shown'].astype(bool)
    limps_df['cards_mucked'] = limps_df['cards_mucked'].astype(bool)
    limps_df['only_blinds'] = limps_df['only_blinds'].astype(bool)
    limps_df['is_shown'] = limps_df['cards_shown'].astype(bool)
    limps_df['is_mucked'] = limps_df['cards_mucked'].astype(bool)
    limps_df['is_open_limp'] = limps_df['limpers_before'] == 0
    limps_df['is_over_limp'] = limps_df['limpers_before'] > 0
    limps_df['is_broadway_combo'] = (
        (limps_df['hi_value'] >= 11)
        & (limps_df['lo_value'] >= 10)
        & (limps_df['hand_group'] != 'Pocket Pair')
    )

    for col in ['faced_raise', 'called_raise', 'raised_after_limp', 'folded_preflop', 'saw_flop']:
        if col not in limps_df.columns:
            if col == 'saw_flop':
                limps_df[col] = True
            else:
                limps_df[col] = False

    limps_df['faced_raise'] = limps_df['faced_raise'].astype(bool)
    limps_df['called_raise'] = limps_df['called_raise'].astype(bool)
    limps_df['raised_after_limp'] = limps_df['raised_after_limp'].astype(bool)
    limps_df['folded_preflop'] = limps_df['folded_preflop'].astype(bool)
    limps_df['saw_flop'] = limps_df['saw_flop'].astype(bool)

    total_events = len(limps_df)
    unique_combos = limps_df['hand_label'].nunique()
    open_count = int(limps_df['is_open_limp'].sum())
    open_pct = open_count / total_events * 100 if total_events else 0.0
    sb_only = int(limps_df['only_blinds'].sum())
    sb_pct = sb_only / total_events * 100 if total_events else 0.0
    shown_pct = limps_df['is_shown'].mean() * 100 if total_events else 0.0
    continued_count = int((limps_df['called_raise'] & limps_df['saw_flop']).sum())
    continued_pct = continued_count / total_events * 100 if total_events else 0.0

    print(f'Loaded {total_events} population limp events with revealed cards.')
    print(f'Unique combos observed: {unique_combos}.')
    print(f'Open limps: {open_count} ({open_pct:.1f}%), over-limps: {total_events - open_count} ({100 - open_pct:.1f}%).')
    print(f'SB completes vs BB only: {sb_only} ({sb_pct:.1f}%).')
    print(f'Cards shown at showdown: {shown_pct:.1f}% (remaining {100 - shown_pct:.1f}% from mucked reveals).')
    print(f'Limped, called a raise, and saw flop: {continued_count} ({continued_pct:.1f}%).')


In [ ]:
import numpy as np

RANK_ORDER = list('AKQJT98765432')

if limps_df.empty:
    combo_matrices = {}
    print('No data available for hand matrix heatmap.')
else:
    def _combo_variations(row):
        if row.hi_rank == row.lo_rank:
            return 6
        return 4 if row.suited else 12

    def _matrix_coords(row):
        if row.hi_rank == row.lo_rank:
            return row.hi_rank, row.lo_rank
        if row.suited:
            return row.hi_rank, row.lo_rank
        return row.lo_rank, row.hi_rank

    limps_df['combo_variations'] = limps_df.apply(_combo_variations, axis=1)
    coords = limps_df.apply(_matrix_coords, axis=1)
    limps_df['matrix_row'] = [c[0] for c in coords]
    limps_df['matrix_col'] = [c[1] for c in coords]
    limps_df['normalized_weight'] = 1 / limps_df['combo_variations']

    counts_matrix = (
        limps_df.groupby(['matrix_row', 'matrix_col'])
        .size()
        .unstack(fill_value=0)
        .reindex(index=RANK_ORDER, columns=RANK_ORDER, fill_value=0)
    )

    normalized_matrix = (
        limps_df.groupby(['matrix_row', 'matrix_col'])['normalized_weight']
        .sum()
        .unstack(fill_value=0)
        .reindex(index=RANK_ORDER, columns=RANK_ORDER, fill_value=0)
    )

    combo_matrices = {
        'raw_counts': counts_matrix,
        'normalized': normalized_matrix,
    }


In [ ]:
if limps_df.empty:
    print('No data available for heatmap plot.')
else:
    import matplotlib.pyplot as plt
    from matplotlib.colors import LinearSegmentedColormap, Normalize
    import numpy as np

    def _render_heatmap(matrix, title, fmt, base_color):
        data = matrix.loc[RANK_ORDER, RANK_ORDER].values
        fig, ax = plt.subplots(figsize=(9, 8))
        max_val = np.nanmax(data) if np.any(data) else 0
        if max_val <= 0:
            max_val = 1.0
        cmap = LinearSegmentedColormap.from_list(f'{base_color}_gradient', ['#ffffff', base_color])
        norm = Normalize(vmin=0, vmax=max_val)
        im = ax.imshow(data, cmap=cmap, norm=norm, interpolation='nearest')
        ticks = range(len(RANK_ORDER))
        ax.set_xticks(ticks)
        ax.set_xticklabels(RANK_ORDER)
        ax.set_yticks(ticks)
        ax.set_yticklabels(RANK_ORDER)
        ax.tick_params(top=True, bottom=True, labeltop=True, labelbottom=True)
        ax.tick_params(left=True, right=True, labelleft=True, labelright=True)
        ax.set_title(title)
        for i in range(len(RANK_ORDER)):
            for j in range(len(RANK_ORDER)):
                value = data[i, j]
                label = '' if value == 0 else fmt.format(value)
                text_color = 'white' if value > 0 and norm(value) >= 0.6 else 'black'
                ax.text(j, i, label, ha='center', va='center', color=text_color, fontsize=8)
        fig.subplots_adjust(top=0.88, right=0.88)
        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        plt.show()

    _render_heatmap(combo_matrices['raw_counts'], 'Limp Frequency by Starting Hand (Raw Count)', '{:.0f}', '#d73027')
    _render_heatmap(combo_matrices['normalized'], 'Limp Frequency by Starting Hand (Normalized per Combo)', '{:.2f}', '#225ea8')


In [ ]:
if limps_df.empty:
    print('No data available for continued limp heatmaps.')
elif 'matrix_row' not in limps_df.columns:
    print('Run the combo matrix cell before generating continued limp heatmaps.')
else:
    continued_df = limps_df[(limps_df['called_raise']) & (limps_df['saw_flop'])]
    reraised_df = limps_df[(limps_df['raised_after_limp']) & (limps_df['saw_flop'])]

    if continued_df.empty:
        print('No limps both called a raise and made it to the flop.')
    else:
        continued_counts = (
            continued_df.groupby(['matrix_row', 'matrix_col'])
            .size()
            .unstack(fill_value=0)
            .reindex(index=RANK_ORDER, columns=RANK_ORDER, fill_value=0)
        )
        continued_normalized = (
            continued_df.groupby(['matrix_row', 'matrix_col'])['normalized_weight']
            .sum()
            .unstack(fill_value=0)
            .reindex(index=RANK_ORDER, columns=RANK_ORDER, fill_value=0)
        )
        _render_heatmap(continued_counts, 'Limp -> Call Raise -> Flop (Raw Count)', '{:.0f}', '#b30059')
        _render_heatmap(continued_normalized, 'Limp -> Call Raise -> Flop (Normalized per Combo)', '{:.2f}', '#1b9e77')

    if reraised_df.empty:
        print('No limps re-raised preflop and made it to the flop.')
    else:
        reraised_counts = (
            reraised_df.groupby(['matrix_row', 'matrix_col'])
            .size()
            .unstack(fill_value=0)
            .reindex(index=RANK_ORDER, columns=RANK_ORDER, fill_value=0)
        )
        reraised_normalized = (
            reraised_df.groupby(['matrix_row', 'matrix_col'])['normalized_weight']
            .sum()
            .unstack(fill_value=0)
            .reindex(index=RANK_ORDER, columns=RANK_ORDER, fill_value=0)
        )
        _render_heatmap(reraised_counts, 'Limp -> Re-raise -> Flop (Raw Count)', '{:.0f}', '#08519c')
        _render_heatmap(reraised_normalized, 'Limp -> Re-raise -> Flop (Normalized per Combo)', '{:.2f}', '#238b45')


In [ ]:
if limps_df.empty:
    print('No data available for position heatmaps.')
else:
    from itertools import cycle
    import matplotlib.pyplot as plt
    from matplotlib.colors import LinearSegmentedColormap, Normalize
    import numpy as np

    position_values = limps_df['position_pre'].dropna().unique().tolist()
    positions = [pos for pos in POSITION_ORDER if pos in position_values and pos != 'BB']
    for pos in position_values:
        if pos == 'BB':
            continue
        if pos not in positions:
            positions.append(pos)

    if not positions:
        print('No positional data available.')
    else:
        base_colors = [
            '#e6550d', '#31a354', '#3182bd', '#756bb1', '#636363', '#e41a1c', '#377eb8',
            '#4daf4a', '#984ea3', '#ff7f00', '#a65628', '#f781bf', '#999999'
        ]
        color_cycle = cycle(base_colors)

        def _build_position_matrices(df):
            counts = (
                df.groupby(['matrix_row', 'matrix_col'])
                .size()
                .unstack(fill_value=0)
                .reindex(index=RANK_ORDER, columns=RANK_ORDER, fill_value=0)
            )
            normalized = (
                df.groupby(['matrix_row', 'matrix_col'])['normalized_weight']
                .sum()
                .unstack(fill_value=0)
                .reindex(index=RANK_ORDER, columns=RANK_ORDER, fill_value=0)
            )
            return counts, normalized

        for pos in positions:
            pos_df = limps_df[limps_df['position_pre'] == pos]
            if pos_df.empty:
                continue
            counts_matrix, normalized_matrix = _build_position_matrices(pos_df)
            base_color = next(color_cycle)
            _render_heatmap(counts_matrix, f"{pos} Limp Frequency (Raw Count)", '{:.0f}', base_color)
            _render_heatmap(normalized_matrix, f"{pos} Limp Frequency (Normalized per Combo)", '{:.2f}', base_color)
